In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from lazypredict.Supervised import LazyClassifier

In [82]:
DATA_DIR = "data_dir"

In [83]:
IMG_SIZE = 224

In [84]:
path_label = []
for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        if filename[-4:] == '.jpg':
            path = os.path.join(dirname, filename)
            label = dirname.split('/')[-1]
            path_label += [(path, label)] 

In [85]:
data=pd.DataFrame(columns=['path','label'])
data['path']=np.array(path_label)[:,0]
data['label']=np.array(path_label)[:,1]
data = data[data['label'] != '.ipynb_checkpoints']

In [ ]:
display(data)

In [ ]:
names=[x for x in sorted(os.listdir(DATA_DIR)) if x != '.DS_Store']
N=list(range(len(names)))
normal_mapping=dict(zip(names,N)) 
reverse_mapping=dict(zip(N,names))       
data['label'] = data['label'].replace(normal_mapping)
paths=data['path'].tolist()
normal_mapping

In [88]:
image_list = []

RESIZE=False

for i,path in enumerate(paths):
    img = plt.imread(path)
    img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    if img.shape == (IMG_SIZE,IMG_SIZE,3) or RESIZE == False:
        img_array = img.flatten()
        image_list.append(img_array)
    elif img.shape == (IMG_SIZE,IMG_SIZE,4):
        img_array = img[:,:,0:3].flatten()
        image_list.append(img_array)
    else:
        print(i,img.shape)
        data=data.drop(index=[i],axis=0)

image_data = np.array(image_list)

In [ ]:
y = data.pop('label')

pca = PCA(n_components=16)
image_data_pca = pca.fit_transform(image_data)
X_reduced_df = pd.DataFrame(image_data_pca)

display(X_reduced_df)
print(X_reduced_df.shape)
X = X_reduced_df

In [90]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

In [ ]:
print(len(X_train))
print(len(X_test))

In [ ]:
clf = LazyClassifier(verbose=0,predictions=True)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

In [ ]:
predictions

In [ ]:
models